In [31]:
import pandas as pd

# Load previously saved files
df_optimal_price = pd.read_csv("C:\\Users\\divya\\OneDrive\\Desktop\\dynamic_pricing_project\\data\\optimal_price_table.csv")
elasticity_summary = pd.read_csv("C:\\Users\\divya\\OneDrive\\Desktop\\dynamic_pricing_project\\data\\elasticity_summary.csv")


In [32]:
df = pd.read_csv("C:\\Users\\divya\\OneDrive\\Desktop\\dynamic_pricing_project\\data\\sales_with_features.csv", parse_dates=['InvoiceDate'])

# Sort by date to get the latest entries
df_sorted = df.sort_values('InvoiceDate', ascending=False)

# Drop duplicates to get the latest price and quantity for each product
df_latest = df_sorted.drop_duplicates(subset='StockCode', keep='first')

# Select only necessary columns
df_latest_prices = df_latest[['StockCode', 'UnitPrice', 'Quantity']].copy()

# Save to CSV
df_latest_prices.to_csv("C:\\Users\\divya\\OneDrive\\Desktop\\dynamic_pricing_project\\data/latest_prices.csv", index=False)

df_latest_prices.head()


,StockCode,UnitPrice,Quantity
57476,22367,1.95,8
242358,22556,1.65,12
17733,22728,3.75,4
16751,22730,3.75,4
56441,22899,2.10,6


In [33]:
# Merge optimal price with current price and quantity
summary = df_optimal_price.merge(df_latest_prices, on='StockCode', how='left')

# Merge with elasticity data
summary = summary.merge(elasticity_summary, on='StockCode', how='left')


In [34]:
# Add current revenue
summary['CurrentRevenue'] = summary['UnitPrice'] * summary['Quantity']

# Revenue gain
summary['RevenueGain'] = summary['SimulatedRevenue'] - summary['CurrentRevenue']

# Revenue gain percentage
summary['RevenueGain(%)'] = (summary['RevenueGain'] / summary['CurrentRevenue']) * 100

# Price change %
summary['PriceChange(%)'] = ((summary['SimulatedPrice'] - summary['UnitPrice']) / summary['UnitPrice']) * 100


In [35]:
summary.describe()

,PriceFactor,SimulatedPrice,SimulatedQuantity,SimulatedRevenue,UnitPrice,Quantity,Avg_Elasticity,CurrentRevenue,RevenueGain,RevenueGain(%),PriceChange(%)
count,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000,2337.000000
mean,0.896500,2.390209,14.156183,21.475052,2.506881,9.362430,-2.926617,15.655618,5.819434,380.436370,18.996468
std,0.158846,2.215669,13.251066,15.156323,2.562661,14.345152,2.314205,36.074080,35.960508,770.465575,78.581568
min,0.800000,0.099934,1.000000,0.883029,0.120000,1.000000,-9.875000,0.120000,-759.050000,-97.459034,-94.767457
25%,0.800000,0.998112,6.000000,10.621333,0.830000,1.000000,-4.657149,2.610000,1.990000,20.369982,-20.000000
50%,0.800000,1.627286,11.000000,17.051139,1.650000,4.000000,-2.881944,7.500000,8.160000,142.126442,-16.938753
75%,0.900000,3.075000,18.000000,28.373072,2.950000,12.000000,-1.043472,16.500000,16.263826,459.712378,22.782373
max,1.200000,15.300000,208.000000,111.666667,13.160000,115.000000,7.083333,833.850000,110.182163,15094.273684,666.666667


In [36]:
final_summary = summary[[
    'StockCode', 'UnitPrice', 'SimulatedPrice', 'PriceChange(%)',
    'Quantity', 'SimulatedQuantity',
    'CurrentRevenue', 'SimulatedRevenue',
    'RevenueGain', 'RevenueGain(%)',
    'Avg_Elasticity'
]].copy()

# Rename for clarity
final_summary.rename(columns={
    'UnitPrice': 'CurrentPrice',
    'Quantity': 'CurrentQuantity',
    'SimulatedPrice': 'OptimalPrice',
    'SimulatedQuantity': 'EstimatedQuantity',
    'SimulatedRevenue': 'EstimatedRevenue',
    'Avg_Elasticity':'Elasticity'
}, inplace=True)

# Round values
final_summary = final_summary.round(2)


In [37]:
final_summary.to_csv("C:\\Users\\divya\\OneDrive\\Desktop\\dynamic_pricing_project\\data/final_price_recommendations.csv", index=False)
final_summary.head()


,StockCode,CurrentPrice,OptimalPrice,PriceChange(%),CurrentQuantity,EstimatedQuantity,CurrentRevenue,EstimatedRevenue,RevenueGain,RevenueGain(%),Elasticity
0,84805B,1.25,1.67,33.33,24,67,30.00,111.67,81.67,272.22,-7.50
1,21398,0.39,0.54,37.27,3,208,1.17,111.35,110.18,9417.28,-8.60
2,90146,3.75,5.00,33.33,9,20,33.75,100.00,66.25,196.30,-7.00
3,23240,4.15,3.32,-20.00,6,30,24.90,99.60,74.70,300.00,-8.39
4,84857C,1.95,2.31,18.39,9,43,17.55,99.27,81.72,465.63,-6.49


In [39]:
old_revenue_col = 'old_revenue'
new_revenue_col = 'new_revenue'

# Calculate totals
total_old_revenue = final_summary['CurrentRevenue'].sum()
total_new_revenue = final_summary['EstimatedRevenue'].sum()

# Calculate net change and percentage change
net_change = total_new_revenue - total_old_revenue
percent_change = (net_change / total_old_revenue) * 100

print(f"Total Old Revenue: ${total_old_revenue:,.2f}")
print(f"Total New Revenue: ${total_new_revenue:,.2f}")
print(f"Net Change in Revenue: ${net_change:,.2f}")
print(f"Percentage Change in Revenue: {percent_change:.2f}%")

Total Old Revenue: $36,587.18
Total New Revenue: $50,187.24
Net Change in Revenue: $13,600.06
Percentage Change in Revenue: 37.17%
